# ReID System - Uruchomienie przez CLI

Najprostszy sposób uruchomienia systemu Re-Identification - bezpośrednio z linii poleceń.

## Setup - Google Colab (opcjonalnie)

In [ ]:
# Tylko jeśli używasz Google Colab
try:
    import google.colab
    IN_COLAB = True

    # Mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')

    %cd master-thesis-reid/reid_system

    # Install dependencies
    !pip install -q ultralytics lap opencv-python torchreid

except:
    IN_COLAB = False
    print("Local run detected.")

## 1. Przetwarzanie klatek z kamery

Przetwarza wszystkie obrazy z folderu `FRAMES_DIR` i zapisuje wyniki do `OUTPUT_DIR`.

In [ ]:
PERSON_MODEL = "/content/drive/MyDrive/reid_models/osnet_x1_0_market1501_preprocessed_k16_lsoftmax+triplet/final_model.pth"
VEHICLE_MODEL = "/content/drive/MyDrive/reid_models/aaver_veri776_preprocessed_k16_lsoftmax/final_model.pth"

FRAMES_DIR = "/content/drive/MyDrive/reid_data/MOT17/train/MOT17-13-DPM/img1"

OUTPUT_DIR = "/content/drive/MyDrive/output_demo_frames_13"

DATABASE_PERSON = "/content/drive/MyDrive/reid_database_person"
DATABASE_VEHICLE = "/content/drive/MyDrive/reid_database_vehicle"

In [ ]:
!python /content/master-thesis-reid/reid_system/run_system.py \
    --frames-dir {FRAMES_DIR} \
    --output-dir {OUTPUT_DIR} \
    --person-model {PERSON_MODEL} \
    --vehicle-model {VEHICLE_MODEL} \
    --person-model-name osnet_x1_0 \
    --vehicle-model-name aaver \
    --person-loss softmax+triplet \
    --vehicle-loss softmax \
    --database-person {DATABASE_PERSON} \
    --database-vehicle {DATABASE_VEHICLE} \
    --confidence 0.8 \
    --device cuda \
    --no-reranking

## 2. Wyświetlenie wyników

Pokaż przetworzone obrazy z rozpoznanymi tożsamościami.

In [ ]:
import matplotlib.pyplot as plt
import cv2
from pathlib import Path

result_files = sorted(list(Path(OUTPUT_DIR).glob('result_*.jpg')) +
                     list(Path(OUTPUT_DIR).glob('result_*.png')))

print(f"Found {len(result_files)} results\n")

for i, result_file in enumerate(result_files[:5]):
    print(f"[{i+1}/{len(result_files)}] {result_file.name}")


    img = cv2.imread(str(result_file))
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Wyświetl
    plt.figure(figsize=(15, 10))
    plt.imshow(img_rgb)
    plt.title(f'Result: {result_file.name}', fontsize=16)
    plt.axis('off')
    plt.tight_layout()
    plt.show()
    print()

## 3. Dodawanie nowej tożsamości do bazy

Dodaj nową osobę lub pojazd do bazy danych.

In [ ]:
!python run_system.py \
    --add-identity person \
    --person-model {PERSON_MODEL} \
    --database {DATABASE_PERSON}

In [ ]:
!python run_system.py \
    --add-identity vehicle \
    --vehicle-model {VEHICLE_MODEL} \
    --database {DATABASE_VEHICLE}

## 4. Zaawansowane opcje

### Zmiana modelu YOLO

In [ ]:
!python run_system.py \
    --frames-dir {FRAMES_DIR} \
    --output-dir {OUTPUT_DIR} \
    --person-model {PERSON_MODEL} \
    --vehicle-model {VEHICLE_MODEL} \
    --database-person {DATABASE_PERSON} \
    --database-vehicle {DATABASE_VEHICLE} \
    --yolo-model yolov8n.pt \
    --device cuda

### Zmiana progu pewności

In [ ]:
!python run_system.py \
    --frames-dir {FRAMES_DIR} \
    --output-dir {OUTPUT_DIR} \
    --person-model {PERSON_MODEL} \
    --vehicle-model {VEHICLE_MODEL} \
    --database-person {DATABASE_PERSON} \
    --database-vehicle {DATABASE_VEHICLE} \
    --confidence 0.9 \
    --device cuda

### Wyłączenie rerankingu (szybsze, mniej dokładne)

In [ ]:
!python run_system.py \
    --frames-dir {FRAMES_DIR} \
    --output-dir {OUTPUT_DIR} \
    --person-model {PERSON_MODEL} \
    --vehicle-model {VEHICLE_MODEL} \
    --database-person {DATABASE_PERSON} \
    --database-vehicle {DATABASE_VEHICLE} \
    --no-reranking \
    --device cuda

### Uruchomienie na CPU (jeśli brak GPU)

In [ ]:
!python run_system.py \
    --frames-dir {FRAMES_DIR} \
    --output-dir {OUTPUT_DIR} \
    --person-model {PERSON_MODEL} \
    --vehicle-model {VEHICLE_MODEL} \
    --database-person {DATABASE_PERSON} \
    --database-vehicle {DATABASE_VEHICLE} \
    --device cpu

## 6. Sprawdzenie bazy danych

In [ ]:
import json
from pathlib import Path

identities_file = Path(DATABASE_PERSON) / 'identities.json'

if identities_file.exists():
    with open(identities_file, 'r') as f:
        identities = json.load(f)

    print(f"Liczba tożsamości w bazie: {len(identities)}\n")
    print(f"{'ID':<15} {'Etykieta':<30} {'Kategoria':<10}")
    print("-" * 60)

    for identity_id, data in identities.items():
        print(f"{identity_id:<15} {data['label']:<30} {data['category']:<10}")
else:
    print("Baza danych jest pusta. Dodaj tożsamości używając --add-identity")